# 01_scrap

## Getting data from The Standard

In [1]:
from bs4 import BeautifulSoup as soup
import requests
from datetime import datetime
import pandas as pd
import json

In [3]:
# thaipbs_url = 'https://workpointtoday.com'
r = requests.get('https://thestandard.co/?s=%E0%B8%AA%E0%B9%88%E0%B8%87%E0%B8%AD%E0%B8%AD%E0%B8%81%E0%B8%97%E0%B8%B8%E0%B9%80%E0%B8%A3%E0%B8%B5%E0%B8%A2%E0%B8%99&search=')
b = soup(r.content,'html.parser')

In [4]:
title = []
urls = []
content = []
published_datetime = []
retrieved_datetime  = []

In [5]:
for news in b.findAll('div', attrs={'class' : 'col-md-8'}):
    titles = news.find('h3').get_text()
    titles = titles.replace('\n','')
    urls.append(news.find('a', href=True)['href'])
    #print(titles)
    title.append(titles)


#print(title)
#print(urls)

In [7]:
for link in urls:
    page = requests.get(link)
    bsobj = soup(page.content,'html.parser')
    for n in bsobj.findAll('div', attrs={'class' : 'entry-content'}):
        dis = n.text
        dis = dis.replace('\n','')
        dis = dis.replace('\xa0','')
        content.append(dis)

    for date in bsobj.findAll('div', attrs={'class' : 'meta-date'}):
        date_p = date.find('span').get_text()
        date_p = datetime.strptime(date_p, '%d.%m.%Y').strftime("%Y-%m-%d")
        published_datetime.append(date_p)
        
    r = (datetime.now()).strftime("%Y-%m-%d")
    retrieved_datetime.append(r)
    
#print(content) 
#print(published_datetime)
#print(retrieved_datetime)

In [10]:
col = ['Title','Publish Date','Retrieve datetime','Content','Source_URL']
table = pd.DataFrame([title,published_datetime,retrieved_datetime,content,urls]).transpose()
table.columns = ['Title','Publish Date','Retrieve datetime','Content','Source_URL']

table.head()

,Title,Publish Date,Retrieve datetime,Content,Source_URL
0,ไทยส่งออกทุเรียนไปจีน ไตรมาสแรกเพิ่มขึ้น 14%,2021-05-05,2022-11-01,เมื่อวานนี้ (4 พฤษภาคม) อรมน ทรัพย์ทวีธรรม อธิ...,https://thestandard.co/thailand-durian-exports...
1,เขาหัวโล้นขยายพื้นที่! มาเลเซียตั้งเป้าส่งออกท...,2019-02-06,2022-11-01,เกษตรกรในมาเลเซียหันมาปลูกทุเรียน แทนการปลูกปา...,https://thestandard.co/malaysia-plans-increase...
2,เรากำลังจะตาย! นโยบายปลอดโควิดของแดนมังกร สะเท...,2022-05-06,2022-11-01,ผู้ปลูกผลไม้ของไทยเป็นกลุ่มล่าสุดที่ได้รับผลกร...,https://thestandard.co/china-covid-free-policy...
3,‘บุฟเฟต์ทุเรียน’ เซ็นทรัลเวิลด์ กลับมาอีกครั้ง...,2022-05-02,2022-11-01,เซ็นทรัล ฟู้ด รีเทล เตรียมจัด ‘บุฟเฟต์ทุเรียน’...,https://thestandard.co/durian-buffet-18-22-may...
4,โลตัสจัด ‘บุฟเฟต์ทุเรียน’ 399 บาท นำรองสาขาราม...,2022-04-30,2022-11-01,โลตัส จัด ‘บุฟเฟต์ทุเรียน’ ราคา 399 บาท นำร่อง...,https://thestandard.co/lotus-s-durian-buffet-3...


In [11]:
table = table.to_dict('records')

In [12]:
with open('thestandard.json','w', encoding='utf8') as json_file:
    json.dump(table, json_file)

## Upload data to DataPlatform

In [13]:
from dsmlibrary.datanode import DataNode

In [14]:
dir_raw_id = 60

In [15]:
datanode = DataNode()

Init DataNode sucessful!


In [16]:
datanode.upload_file(directory_id=dir_raw_id, file_path='thestandard.json')

  0%|          | 0.00/195k [00:00<?, ?B/s]

{'id': 324,
 'msg': 'file thestandard.json create sucessful',
 'path': 'supitchaya/DSI310-04/rawData/thestandard.json',
 'uploaded': True}